chromedriver

http://chromedriver.storage.googleapis.com/index.html

Está la librería? si no instalarla. Ojo, tras instalarla es buena idea hacer Kernel+Restart y volver a ejecutar este código

In [1]:
import sys
import os.path
from subprocess import check_call
import importlib
import os

modules = ["selenium","pymongo","pprint"]
print("Checking required modules")
for m in modules:
    torch_loader = importlib.util.find_spec(m)
    if torch_loader is not None:
        print(m," found")
    else:
        print(m," not found, installing")
        if 'google.colab' in sys.modules:
            check_call(["pip", "install", "-q", m])
        else:
            check_call([sys.executable, "-m", "pip", "install", "--user", m])
print("Done!")

Checking required modules
selenium  found
pymongo  found
pprint  found
Done!


Conectamos a MongoDB 

In [2]:
from pprint import pprint # para mostrar los json bonitos
from pymongo import MongoClient

# conexión a mongo
servidor = 'localhost'
puerto = 27017

basededatos = 'mirko'

client = MongoClient(servidor, puerto)
db = client[basededatos]


Cargamos la colección user, ojo porque esto borra esta coleccion
**OJO**: esto destruye la colección anterior

In [ ]:
import json # para transformar la línea leida (string) a json
import urllib.request # para leer de la URL línea a línea


def carga_desde_fichero(db,fichero,coleccion):
    db[coleccion].drop() # la borramos
    exito,error = 0,0
    # cargamos los datos desde el fichero
    try:
        with urllib.request.urlopen(fichero) as f:
        
            for line in f:
                line2 = line.decode("UTF-8").replace("$","")
                res = db[coleccion].insert_one(json.loads(line2))
                if res.acknowledged:
                    exito+=1
                else:
                    error+=1
        print(f"Colección {coleccion}: {exito} documentos cargados con éxito y {error} errores")
    except urllib.error.URLError as e:
        print(e.reason)
    return
   


fichero_user = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/userdir.json"
carga_desde_fichero(db,fichero_user,"user")


Arrancar Chrome; recordar cambiar antes el camino al fichero

In [11]:

chromedriver = r"C:\Users\nolat\chromedriver.exe"
chromedriver = r"C:\hlocal\chromedriver.exe"

import os

from selenium import webdriver  
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(executable_path=chromedriver,options=chrome_options)

Descomentar y ejecutar esto solo para resetear "renta" si se considera necesario

In [ ]:
#db.user.update_many({}, {"$unset":{"renta":1}})

In [ ]:
db.user.create_index([("renta",1)])

In [13]:
from collections import Counter
from selenium.webdriver.common.keys import Keys
import time

def info(driver,calle):
    url="https://widgets.elpais.com/mapbox/2021/04/renta-secciones/"
    driver.get(url) # abre
    input = driver.find_element_by_tag_name("input") # solo hay 1 por fortuna
    input.clear()
    input.send_keys(calle) # aquí poner la dirección
    time.sleep(2)
    input.send_keys(Keys.ENTER)
    time.sleep(10)
    panel = driver.find_element_by_id("panel-info")
    return panel.text

def quita_municipality(dir,doc):
    if "address" in doc["direccion"] and 'municipality' in doc["direccion"]["address"]:
        print("Quitando municipality")
        dir = dir.replace(doc["direccion"]["address"]["municipality"]+",","")
    return dir

def quita_barra(dir):
    if "/" in dir:
        #print("Quitando barra de ",dir)
        pos = dir.index("/")
        #print("/ pos",pos)
        if "," in dir[pos:]:
            poscoma = dir[pos:].index(",")
            #print("pos coma",poscoma)
            dir = dir[:pos]+dir[(pos+poscoma):]
            #print("dir cambiado",dir)
            dir = quita_barra(dir) # por si hay más barras
    return dir

# quitar las direcciones antiguas
# ahora tenemos que añadir a cada usuario un campo direccion
cursor = db.user.find({"$or": [{"renta":{"$exists":False}},{"renta":""}]})
borrados=0
for doc in cursor:
    print(doc["_id"])
    #print(doc["direccion"])
    dir=doc["direccion"]["display_name"]
    print("Dir antes de preprocesado:",dir)
    
    ## quitamos municipality
    dir = quita_municipality(dir,doc)
    
    ## quitamos desde '/' hasta ',' (sin incluir)? os lo dejo a vosotras
    dir = quita_barra(dir)
    print("Dir después preprocesado: ", dir)
    renta = info(driver,dir)
    print("Renta: ")
    print(renta)
    db.user.update_one({"_id":doc["_id"]}, {"$set":{"renta":renta}})
    print("*"*80)


17581694
Dir antes de preprocesado: Carretera de LE-711 (Vega de Espinareda) por Balouta a L.P. Lugo, Rao, Navia de Suarna, Os Ancares, León, Galicia, España
Dir después preprocesado:  Carretera de LE-711 (Vega de Espinareda) por Balouta a L.P. Lugo, Rao, Navia de Suarna, Os Ancares, León, Galicia, España
Renta: 

********************************************************************************
40604112
Dir antes de preprocesado: Carrer Joan de Joanes, Platja de Tavernes de la Valldigna, Tavernes de la Valldigna, la Safor, València / Valencia, Comunitat Valenciana, 46760, España
Dir después preprocesado:  Carrer Joan de Joanes, Platja de Tavernes de la Valldigna, Tavernes de la Valldigna, la Safor, València , Comunitat Valenciana, 46760, España
Renta: 

********************************************************************************
72269408
Dir antes de preprocesado: Camí del Trabucador, Sant Carles de la Ràpita, Montsià, Tarragona, Catalunya, 43540, España
Quitando municipality
Dir de

KeyboardInterrupt: 

Por si hemos visto error en alguno concreto y queremos quitarle renta

In [ ]:
db.user.update_one({"_id":"215661586"}, {"$unset":{"renta":1}})